# Symbolic Computation Speedups

This is a simple example of how symbolic computation can yield large reductions in the amount of computation.

In [ ]:
# Setup
import symforce

symforce.set_symbolic_api("sympy")

import symforce.symbolic as sf
from symforce.notebook_util import display

# Autodiff vs Symbolic diff

### Multiplying three matrices (for example to chain rule Jacobians)

In [ ]:
A = sf.Matrix.zeros(5, 5).symbolic("A")
B = sf.Matrix.zeros(5, 3).symbolic("B")
C = sf.Matrix.zeros(3, 3).symbolic("C")
display(A, B, C)

In [ ]:
display(A * B * C)

In [ ]:
# Normal matrix multiplication (can happen with SIMD)
sf.count_ops(A * B * C)

### The same with typical sparsity patterns

In [ ]:
A = sf.Matrix.diag(sf.symbols("A:5"))
A[0, 4] = sf.Symbol("A5")
B = sf.Matrix.zeros(5, 3)
B[:3, :3] = sf.Matrix.diag(sf.symbols("B:3"))
B[3, 0] = sf.Symbol("B3")
B[3, 1] = sf.Symbol("B3")
B[3, 2] = sf.Symbol("B3")
B[4, 1] = sf.Symbol("B4")
C = sf.M(sf.Rot3.hat(sf.symbols("C:3")))
display(A, B, C)

In [ ]:
display(A * B * C)

In [ ]:
# Direct computation of result
sf.count_ops(A * B * C)

In [ ]:
# Looking at the sub-expressions
intermediates, output = sf.cse(A * B * C)

num_operations = 0
for lhs, rhs in intermediates:
    display(sf.sympy.Eq(lhs, rhs))
    num_operations += sf.count_ops(rhs)

output_mat = sf.Matrix53(output)
num_operations += sf.count_ops(output_mat)
display(output_mat)

In [ ]:
# With common sub-expression elimination
display(num_operations)

In [ ]:
# Code generation
for lhs, rhs in intermediates:
    print(f"float {lhs} = {rhs};")

print("")
for i, out in enumerate(output):
    print(f"out[{i}] = {out};")

Further points:

* simple example, entries are leaves
* only three matrices deep
* applies to other domains, not just derivatives + optimization
* not considering dynamic allocation (matrices on the heap), pointer chasing
* not considering you had to write the jacobians in the first place, test, debug